In [1]:
import pandas as pd

# Explication du Code : Génération de deux datasets sur les discussions autour de l'agriculture biologique

Ce document explique le but et la logique derrière le code fourni. Celui-ci vise à créer deux datasets destinés à analyser les discussions sur l'agriculture biologique en France, mois par mois. Voici les détails des deux datasets générés :

## Dataset 1 : Valeur Quantitative (Importance des discussions)

- **Description** : Ce dataset associe à chaque mois une valeur entre 0 et 1, appelée "valeur quantitative". Elle représente l'importance relative des discussions autour de l'agriculture biologique durant ce mois.
- **Méthode de calcul** : 
  - La valeur est obtenue en faisant la **moyenne des trois indicateurs suivants** :
    1. **Résultats de Pytrends** : Une mesure de l'intérêt des recherches sur l'agriculture biologique.
    2. **Nombre d'articles publiés dans *Le Monde*** : Normalisé pour obtenir une valeur relative.
    3. **Nombre d'articles publiés dans *LSA*** : Également normalisé.
  - Ces trois métriques sont pondérées de manière égale pour produire une valeur globale entre 0 et 1.

## Dataset 2 : Valeur Qualitative (Sentiment des discussions)

- **Description** : Ce dataset associe à chaque mois une valeur entre 0 et 1, appelée "valeur qualitative". Elle reflète le caractère positif ou négatif des discussions sur l'agriculture biologique durant ce mois.
- **Méthode de calcul** :
  - La valeur est obtenue en calculant la **moyenne des scores de sentiment analysis** pour tous les articles pertinents publiés durant le mois.
  - Les articles sont extraits de trois sources principales :
    1. *Le Monde*
    2. *HuffPost*
    3. *LSA*
  - Chaque article est analysé à l'aide d'un outil de sentiment analysis pour obtenir une note comprise entre -1 (très négatif) et 1 (très positif), convertie sur une échelle de 0 à 1 pour uniformiser les résultats.

## Objectifs du Projet

1. **Comprendre les tendances** : Identifier les périodes où l'agriculture biologique a été un sujet particulièrement discuté.
2. **Analyser les sentiments** : Déterminer si ces discussions étaient globalement positives ou négatives au fil du temps.
3. **Synthétiser les données** : Fournir une base de données claire et exploitable pour d'autres analyses ou visualisations.

Le code qui suit implémente ces logiques et produit les deux datasets décrits ci-dessus.

In [101]:
# lecture des csv 
result_pytrend = pd.read_csv('data_quantitative.csv')
data_LSA = pd.read_csv('data_LSA_archive.csv')
data_leMonde = pd.read_csv('data_Lemonde.csv')
data_huffpost = pd.read_csv('data_huffpost.csv')

## Création du dataset quantitatif

In [78]:
# Conversion de Month_Year dans result (au format 01-2000)
result_pytrend['Month_Year'] = pd.to_datetime(result_pytrend['Month_Year'], format='%m-%Y')

In [82]:
# Conversion de Publication_Date dans data_LSA (au format 2024-11-26 09:00:00)
data_LSA['Publication_Date'] = pd.to_datetime(data_LSA['Publication_Date'], errors='coerce')

# Extraire Month_Year de Publication_Date pour avoir une colonne 'Month_Year'
data_LSA['Month_Year'] = data_LSA['Publication_Date'].dt.to_period('M').dt.to_timestamp()

# Compter le nombre d'articles par mois dans data_LSA
articles_per_month = data_LSA.groupby('Month_Year').size()

# Trouver le nombre maximum d'articles dans un mois
max_articles_in_month = articles_per_month.max()

# Fusionner les données avec result (en utilisant Month_Year comme clé de fusion)
result_pytrend['Month_Year'] = pd.to_datetime(result_pytrend['Month_Year'], format='%Y-%m')  # Assurer que Month_Year est en datetime

# Fusionner avec les articles LSA par mois
result_pytrend = result_pytrend.merge(articles_per_month_df, on='Month_Year', how='left')

# Normaliser le ratio : le mois avec le plus grand nombre d'articles aura un ratio de 1
result_pytrend['LSA_Articles_Ratio'] = articles_per_month / max_articles_in_month

result_pytrend['LSA_Articles_Ratio'] = result_pytrend['LSA_Articles_Ratio'].fillna(0.5)

In [95]:
# Convertir la colonne 'Date' en format datetime en spécifiant le format 'dd-mm-yyyy'
data_leMonde['Date'] = pd.to_datetime(data_leMonde['Date'], format='%d-%m-%Y')

# Extraire le mois et l'année de 'Date' pour avoir une colonne 'Month_Year'
data_leMonde['Month_Year'] = data_leMonde['Date'].dt.to_period('M').dt.to_timestamp()

In [80]:
# Compter le nombre d'articles par mois dans data_LSA
articles_per_month = data_leMonde.groupby('Month_Year').size()

# Trouver le nombre maximum d'articles dans un mois
max_articles_in_month = articles_per_month.max()

# Fusionner avec les articles LSA par mois
result_pytrend = result_pytrend.merge(articles_per_month_df, on='Month_Year', how='left')

# Normaliser le ratio : le mois avec le plus grand nombre d'articles aura un ratio de 1
result_pytrend['Lemonde_Articles_Ratio'] = articles_per_month / max_articles_in_month

result_pytrend['Lemonde_Articles_Ratio'] = result_pytrend['Lemonde_Articles_Ratio'].fillna(0.5)

In [83]:
result_pytrend

,Month_Year,agriculture bio,Article_Count_x,Lemonde_Articles_Ratio,Article_Count_y,LSA_Articles_Ratio
0,2000-01-01,0.5,NaN,0.5,NaN,0.5
1,2000-02-01,0.5,NaN,0.5,NaN,0.5
2,2000-03-01,0.5,NaN,0.5,NaN,0.5
3,2000-04-01,0.5,NaN,0.5,NaN,0.5
4,2000-05-01,0.5,NaN,0.5,NaN,0.5
...,...,...,...,...,...,...
295,2024-08-01,33.0,NaN,0.5,NaN,0.5
296,2024-09-01,33.0,4.0,0.5,4.0,0.5
297,2024-10-01,33.0,11.0,0.5,11.0,0.5
298,2024-11-01,33.0,10.0,0.5,10.0,0.5


In [86]:
# Calculer la moyenne des trois colonnes : agriculture bio, LSA_Articles_Ratio, LeMonde_Articles_Ratio
result_pytrend['Average'] = result_pytrend[['agriculture bio', 'LSA_Articles_Ratio', 'Lemonde_Articles_Ratio']].mean(axis=1)

# Créer un nouveau DataFrame avec uniquement 'Month_Year' et la moyenne calculée
final_result = result_pytrend[['Month_Year', 'Average']]

# Sauvegarder le nouveau DataFrame dans un CSV
final_result.to_csv('data_quantitative.csv', index=False)

print("Le fichier CSV avec la moyenne calculée a été créé avec succès.")


Le fichier CSV avec la moyenne calculée a été créé avec succès.


In [ ]:
## Création du dataset qualitatif

In [92]:
# Convertir la colonne 'Date' en format datetime en spécifiant le format approprié
data_huffpost['Date'] = pd.to_datetime(data_huffpost['Date'], format='%d/%m/%Y %H:%M')

# Extraire le mois et l'année de 'Date' pour avoir une colonne 'Month_Year'
data_huffpost['Month_Year'] = data_huffpost['Date'].dt.to_period('M').dt.to_timestamp()


In [103]:
# Calculer la moyenne des Sentiment_Score pour chaque mois dans chaque fichier
# Pour data_LSA
lsa_avg_sentiment = data_LSA.groupby('Month_Year')['Sentiment_Score'].mean().reset_index()

# Pour data_LeMonde
lemonde_avg_sentiment = data_LeMonde.groupby('Month_Year')['Average_Sentiment_Score'].mean().reset_index()

# Pour data_huffpost
huffpost_avg_sentiment = data_huffpost.groupby('Month_Year')['Average_Sentiment_Score'].mean().reset_index() 

KeyError: 'Month_Year'

In [ ]:
# Fusionner les résultats
# Fusionner les 3 DataFrames sur la colonne 'Month_Year'
merged = pd.merge(lsa_avg_sentiment, lemonde_avg_sentiment, on='Month_Year', how='outer', suffixes=('_LSA', '_LeMonde'))
merged = pd.merge(merged, huffpost_avg_sentiment, on='Month_Year', how='outer')
merged.rename(columns={'Average_Sentiment_Score': 'Average_Sentiment_Score_Huffpost'}, inplace=True)

# Calculer la moyenne des 3 moyennes de sentiment par mois
merged['Average_Sentiment_Score'] = merged[['Sentiment_Score_LSA', 'Average_Sentiment_Score_LeMonde', 'Average_Sentiment_Score_Huffpost']].mean(axis=1)

# Créer le fichier CSV final
final_result = merged[['Month_Year', 'Average_Sentiment_Score']]

# Sauvegarder le CSV final
final_result.to_csv('data_qualitatif.csv', index=False)

print("Le fichier final_sentiment_score_by_month.csv a été créé avec succès.")


KeyError: 'Month_Year'